# SpaceX Falcon 9 SQL EDA

## 1. Introduction

This notebook demonstrates exploratory data analysis using SQL on the SpaceX Falcon 9 dataset.
We will explore landing outcomes, success rates across different features, and relationships between weather, payload, and first-stage landing success.


## 2. Setting up the SQL Environment

In this section, we’ll prepare the environment to run SQL queries inside the notebook. We’ll:  
1. Install required libraries.  
2. Import Python modules.  
3. Load the `ipython-sql` extension.  
4. Configure an SQLite database.  
5. Load our dataset into the database as a table.  

This will allow us to use `%%sql` cell magics to run queries against our dataset.  

### 2.1 Install Required Libraries  
We install the packages needed for SQL integration:  
- **sqlalchemy**: database connection support.  
- **pandas**: data handling and CSV loading.  
- **ipython-sql**: SQL cell magics (`%%sql`).  
- **prettytable**: clean tabular formatting for query results.

In [1]:
!pip install -q sqlalchemy==1.3.9
!pip install -q pandas
!pip install -q ipython-sql
!pip install -q ipython-sql prettytable

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires SQLAlchemy<3,>=1.4, but you have sqlalchemy 1.3.9 which is incompatible.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.3.9 which is incompatible.
google-adk 1.13.0 requires sqlalchemy<3.0.0,>=2.0, but you have sqlalchemy 1.3.9 which is incompatible.
alembic 1.16.5 requires SQLAlchemy>=1.4.0, but you have sqlalchemy 1.3.9 which is incompatible.


### 2.2 Import Python Modules

Next, we import the libraries we’ll use for database connections and data handling.

In [2]:
import csv, sqlite3
import prettytable
import pandas as pd

### 2.3 Load the ipython-sql Extension

The ipython-sql extension enables us to use %%sql magic commands in Jupyter notebooks.

In [3]:
%load_ext sql

### 2.4 Configure SQLite Database

We create a local SQLite database (my_data1.db) and set PrettyTable’s default style for cleaner SQL output.

In [4]:
prettytable.DEFAULT = 'DEFAULT'
con = sqlite3.connect("my_data1.db")
cur = con.cursor()

### 2.5 Connect %sql Magic to the Database

Here, we bind the %%sql magic to our SQLite database.

In [5]:
%sql sqlite:///my_data1.db

### 2.6 Load Dataset into SQL Table

Finally, we load our CSV dataset (dataset_part_2.csv) into the database as a table named SPACEXTBL.

In [6]:
df = pd.read_csv("dataset_part_2.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

90

## 3. Landing Outcomes Overview  

Now that the dataset is loaded into SQL, we begin by exploring the distribution of landing outcomes.  
These queries help us understand how often landings were successful vs. unsuccessful, and provide a baseline success rate.  

In [7]:
# Count of each landing outcome
%%sql SELECT Outcome, COUNT(*) AS Total
FROM SPACEXTBL
GROUP BY Outcome
ORDER BY Total DESC;

 * sqlite:///my_data1.db
Done.


Outcome,Total
True ASDS,41
None None,19
True RTLS,14
False ASDS,6
True Ocean,5
None ASDS,2
False Ocean,2
False RTLS,1


👉 This query shows the distribution of landing outcomes. True ASDS is the most common outcome.

In [8]:
# Overall success rate (binary Class)
%sql SELECT AVG(Class) AS OverallSuccessRate FROM SPACEXTBL;

 * sqlite:///my_data1.db
Done.


OverallSuccessRate
0.6666666666666666




👉 The overall success rate (`AVG(Class)`) is 67%.

## 4. Landing Success by Launch Site  

Different launch sites may have different conditions, infrastructure, or mission profiles.  
Here we compare landing success rates across sites.  

In [9]:
%%sql SELECT LaunchSite,
       AVG(Class) AS SuccessRate,
       COUNT(*) AS NumLaunches
FROM SPACEXTBL
GROUP BY LaunchSite
ORDER BY SuccessRate DESC;

 * sqlite:///my_data1.db
Done.


LaunchSite,SuccessRate,NumLaunches
KSC LC 39A,0.7727272727272727,22
VAFB SLC 4E,0.7692307692307693,13
CCSFS SLC 40,0.6,55


👉 Launch sites differ in geography, infrastructure, and mission type.

👉 Launch sites generally have a similar success rate, between 60% and77%.

## 5. Landing Success by Orbit

In [10]:
%%sql SELECT Orbit,
       AVG(Class) AS SuccessRate,
       COUNT(*) AS NumLaunches
FROM SPACEXTBL
GROUP BY Orbit
ORDER BY SuccessRate DESC;

 * sqlite:///my_data1.db
Done.


Orbit,SuccessRate,NumLaunches
SSO,1.0,5
HEO,1.0,1
GEO,1.0,1
ES-L1,1.0,1
VLEO,0.8571428571428571,14
LEO,0.7142857142857143,7
PO,0.6666666666666666,9
MEO,0.6666666666666666,3
ISS,0.6190476190476191,21
GTO,0.5185185185185185,27


👉 Different orbits require different flight profiles.  
👉 Some orbits are consistently reliable, while others show lower success rates due to higher mission difficulty.  
👉 This helps identify “risky” orbits for landings.

## 6. Landing Success by Payload Category

In [11]:
%%sql SELECT PayloadCategory,
       AVG(Class) AS SuccessRate,
       COUNT(*) AS NumLaunches
FROM SPACEXTBL
GROUP BY PayloadCategory
ORDER BY PayloadCategory;

 * sqlite:///my_data1.db
Done.


PayloadCategory,SuccessRate,NumLaunches
Heavy,0.8214285714285714,28
Light,0.6333333333333333,30
Medium,0.5625,32


👉 Heavier payloads generally make landings harder.

👉 However, the highest succes rate of payloads is "Heavy".

## 7. Landing Success by Temperature Category

In [12]:
%%sql SELECT TempCategory,
       AVG(Class) AS SuccessRate,
       COUNT(*) AS NumLaunches
FROM SPACEXTBL
GROUP BY TempCategory
ORDER BY TempCategory;

 * sqlite:///my_data1.db
Done.


TempCategory,SuccessRate,NumLaunches
Cold,0.5,2
Hot,0.7058823529411765,34
Moderate,0.6481481481481481,54


👉 Extreme heat or cold can affect rocket materials and systems.  
👉 Here we check if temperature extremes reduce landing success.  
👉 However, there is very little difference between “Hot” and "Moderate" temperatures.

## 8. Landing Success by Wind Category

In [13]:
%%sql SELECT WindCategory,
       AVG(Class) AS SuccessRate,
       COUNT(*) AS NumLaunches
FROM SPACEXTBL
GROUP BY WindCategory
ORDER BY WindCategory;

 * sqlite:///my_data1.db
Done.


WindCategory,SuccessRate,NumLaunches
High,0.6,5
Low,0.59375,32
Moderate,0.7169811320754716,53


👉 High winds increase landing difficulty.  
👉 This query shows that "Moderate" winds have the highest success rates.  
👉 However, there seems to be not much difference between "High" and "Low" winds.

## 9. Combined Weather Effects

In [14]:
# Success rate by Temp + Wind category
%%sql SELECT TempCategory, WindCategory,
       AVG(Class) AS SuccessRate,
       COUNT(*) AS NumLaunches
FROM SPACEXTBL
GROUP BY TempCategory, WindCategory
ORDER BY TempCategory, WindCategory;

 * sqlite:///my_data1.db
Done.


TempCategory,WindCategory,SuccessRate,NumLaunches
Cold,Low,1.0,1
Cold,Moderate,0.0,1
Hot,High,1.0,1
Hot,Low,0.6666666666666666,12
Hot,Moderate,0.7142857142857143,21
Moderate,High,0.5,4
Moderate,Low,0.5263157894736842,19
Moderate,Moderate,0.7419354838709677,31


👉 Weather rarely acts alone.  
👉 This combined query checks the interaction between temperature and wind.  
👉 Patterns here reveal whether certain weather combinations (e.g., "Hot" + "Windy") are especially problematic. However, the lowest success rate with a significant number of launches is "Moderate" and "Low", which is unexpected.

## 10. Advanced Analysis  

In this section, we explore more complex queries combining multiple features (payload, orbit, weather, launch site).  
These give us deeper insights into patterns that may not be visible from single-feature breakdowns.  

### 10.1 High Payload & Weather Impact  
Here we check whether heavy payloads combined with extreme weather conditions reduce success rates.  

In [15]:
%%sql SELECT PayloadCategory, TempCategory, WindCategory,
       AVG(Class) AS SuccessRate,
       COUNT(*) AS NumLaunches
FROM SPACEXTBL
GROUP BY PayloadCategory, TempCategory, WindCategory
ORDER BY PayloadCategory, TempCategory, WindCategory;

 * sqlite:///my_data1.db
Done.


PayloadCategory,TempCategory,WindCategory,SuccessRate,NumLaunches
Heavy,Cold,Low,1.0,1
Heavy,Hot,Low,0.7142857142857143,7
Heavy,Hot,Moderate,1.0,4
Heavy,Moderate,High,1.0,1
Heavy,Moderate,Low,0.7,10
Heavy,Moderate,Moderate,1.0,5
Light,Hot,Low,1.0,2
Light,Hot,Moderate,0.6666666666666666,6
Light,Moderate,High,0.5,2
Light,Moderate,Low,0.4,5


👉 Combines payload with weather categories.  
👉 Useful for spotting whether “heavy payload + extreme weather” scenarios are especially failure-prone.

### 10.2 Most Reliable Orbit + Launch Site Combinations  
We look for orbit + site pairs with the highest landing success rates.  

In [16]:
%%sql SELECT LaunchSite, Orbit,
       AVG(Class) AS SuccessRate,
       COUNT(*) AS NumLaunches
FROM SPACEXTBL
GROUP BY LaunchSite, Orbit
ORDER BY SuccessRate DESC
LIMIT 10;

 * sqlite:///my_data1.db
Done.


LaunchSite,Orbit,SuccessRate,NumLaunches
CCSFS SLC 40,ES-L1,1.0,1
CCSFS SLC 40,GEO,1.0,1
CCSFS SLC 40,HEO,1.0,1
CCSFS SLC 40,SSO,1.0,1
KSC LC 39A,ISS,1.0,5
KSC LC 39A,LEO,1.0,2
VAFB SLC 4E,SSO,1.0,4
CCSFS SLC 40,VLEO,0.8888888888888888,9
KSC LC 39A,VLEO,0.8,5
CCSFS SLC 40,MEO,0.6666666666666666,3


👉 Identifies launch site + orbit pairs with the highest reliability.  
👉 These combinations are the “safest bets” for future missions.  
👉 Ranking them highlights best practices.

### 10.3 Rare Conditions  
This query highlights unusual feature combinations with very few launches.  

In [17]:
# Identify combinations with very few launches (less than 3)
%%sql SELECT TempCategory, WindCategory, PayloadCategory,
       COUNT(*) AS NumLaunches
FROM SPACEXTBL
GROUP BY TempCategory, WindCategory, PayloadCategory
HAVING NumLaunches < 3;

 * sqlite:///my_data1.db
Done.


TempCategory,WindCategory,PayloadCategory,NumLaunches
Cold,Low,Heavy,1
Cold,Moderate,Medium,1
Hot,High,Medium,1
Hot,Low,Light,2
Moderate,High,Heavy,1
Moderate,High,Light,2
Moderate,High,Medium,1


👉 Finds unusual combinations (few launches).  
👉 These rare cases are important: limited data may hide risks or opportunities.  
👉 Engineers might treat these cautiously due to lack of evidence.

## ✅ 11. Conclusion

This notebook focused on **exploratory data analysis with SQL** to uncover insights into Falcon 9 first-stage landing outcomes:  

- **Landing success rates**: Varied strongly by orbit type and slightly by launch site.  
- **Weather impact**: Temperature and wind conditions influenced landing success, especially when combined with payload mass.  
- **Multi-feature analysis**: SQL queries revealed patterns not visible in single-feature summaries.  

✅ **Skills demonstrated**: SQL querying for aggregation, grouping, filtering, and multi-feature analysis; integrating SQL with Python via `ipython-sql`; and managing datasets within an SQLite database.  

This notebook builds on the **Data Wrangling and Preparation** notebook and provides a bridge to the upcoming **visual EDA** and **predictive modeling** notebooks in the portfolio.  
  
